# Лабораторная работа №3

### Выполнила студентка Атаулина Рената группы БВТ1901

### Задание №1: Кнута-Морриса-Пратта

In [1]:
def prefix(s):
    #  k  наибольшего собственного префикса подстроки s[0:i], в который одновременно является суффиксом этой подстроки
    p = [0] * len(s)
    for i in range(1, len(s)):
        k = p[i - 1]
        while k > 0 and s[k] != s[i]:
            k = p[k - 1]
        if s[k] == s[i]:
            k += 1
        p[i] = k
    return p
def KMP(s, w):
    
      #  s: Строка, по которой будет произведен поиск
      #  w: Строка, которую будем искать 
    A = []
    k = 0
        
    p = prefix(s)

    for i in range(len(s)):
        while k > 0 and s[i] != w[k]:
            k = p[k-1]
        if s[i] == w[k]:
            k += 1
        if k == len(w):
            A.append((i - len(w) + 1, i+1))
            k = p[k-1]

    return A
s =  "aabaabaaaabaabaaab"
w = "aabaa"
print("")
print(KMP(s, w))


[(0, 5), (3, 8), (8, 13), (11, 16)]


### Задание №1: Бойера-Мура

In [2]:
def preprocess(w):
    """ Строит таблицу смещений """
    T = [len(w)]*256
    for i in range(len(w) - 1):
        T[ord(w[i])] = len(w) - 1 - i
    return T

def BM(s, w):
    """ Алгоритм Бойера-Мура """
    A = []
    T = preprocess(w)
    skip = 0
    while len(s) - skip >= len(w):
        if s[skip:skip + len(w)] == w:
            A.append((skip, skip + len(w)))
        skip += T[ord(s[skip + len(w) - 1])]
    return A
s =  "aabaabaaaabaabaaab"
w = "aabaa"
print("")
print(BM(s, w))


[(0, 5), (3, 8), (8, 13), (11, 16)]


### Время работы алгоритмов

In [3]:
import timeit
s =  "aabaabaaaabaabaaab"
w = "aabaa"
print("Время работы алгоритма 1  " + str(timeit.timeit("KMP(s,w)", number=1, globals=globals())))
print("Время работы алгоритма 2  " + str(timeit.timeit("BM(s,w)", number=1, globals=globals())))

Время работы алгоритма 1  4.7799999947528704e-05
Время работы алгоритма 2  2.6299999944967567e-05


### Задание №2: Пятнашки

In [4]:
from copy import deepcopy

class PuzzleGame:
    def __init__(self, initial_state):
        self.initial_state = initial_state
        self.matrix = []

        self.MOVE = {
            'UP'    :   (-1, 0),
            'DOWN'  :   (1, 0), 
            'LEFT'  :   (0, -1),
            'RIGHT' :   (0, 1)
            }

    def get_matrix(self):
        matrix = ''
        for row in self.matrix:
            matrix += str(row) + '\n'

        return matrix

    def __str__(self):
        return self.get_matrix()

    def check_moves(self, prev_move): # контроль перемещений
        if prev_move == None:
            return tuple(self.MOVE.keys())

        if prev_move == 'UP':
            return ('UP', 'LEFT', 'RIGHT')

        if prev_move == 'DOWN':
            return ('DOWN', 'LEFT', 'RIGHT')

        if prev_move == 'LEFT':
            return ('UP', 'DOWN', 'LEFT')

        if prev_move == 'RIGHT':
            return ('UP', 'DOWN', 'RIGHT')

    def move_allowed(self, place, move): # можно ли переместить
        row, col = self.new_place(place, move)
        if row > 0 and row < 5 and col > 0 and col < 5:
            return True

        else:
            return False

    def new_place(self, place, move): # новая координата нуля
        return (place[0]+move[0], place[1]+move[1])

    def apply_move(self, place, new_place): # копируем матрицу
        o_r, o_c = place
        n_r, n_c = new_place

        new_matrix = deepcopy(self.matrix) # копируем массив

        new_matrix[n_r-1][n_c-1] = new_matrix[o_r-1][o_c-1]
        new_matrix[o_r-1][o_c-1] = self.matrix[n_r-1][n_c-1]

        return new_matrix

    def value_replaced(self, new_place):
        n_r, n_c = new_place
        return self.matrix[n_r-1][n_c-1]

    def distance(self, prev_place, new_place):
        x1, y1 = prev_place
        x2, y2 = new_place
        d = abs(x2 - x1) + abs(y2 - y1)

        return d

    def count_dissimilar(self, matrix): # кол-во эл-в не на своем месте
        dissmilar = 0
        count = 1
        for row in matrix:
            for col in row:
                if col and col is not count:
                    dissmilar += 1

                count += 1

        return dissmilar

    def generate_matrix(self, initial_state):
        return [initial_state[i:i+4] for i in range(0, len(initial_state), 4)]

    def generate_numbers(self, grid_size):
        return [i for i in range(1, grid_size)] + [None]

    def is_solvable(self):
        self.place = self.get_block_place(block=None, matrix=self.matrix)
        result = self.sum_less_i() + self.b(block=None, matrix=self.matrix)

        return (result % 2 == 0)

    def get_block_place(self, block, matrix): # возвращает координаты None
        for row in range(0, 4):
            for col in range(0, 4):
                if (matrix[row][col] is block):
                    return (row+1, col+1)
                
    def b(self, block, matrix): 
        for row in range(0, 4):
            for col in range(0, 4):
                if (matrix[row][col] is block):
                    return (row+1)
                
    def sum_less_i(self):
        sumission = 0
        for num in range(0, 16):
            if self.initial_state[num] is None:
             #   sumission += (16 - (num + 1))
                continue
            
            for current in range(num + 1, 16):
                if (self.initial_state[current] is None):
                    continue

                if (self.initial_state[num] > self.initial_state[current]):
                    sumission += 1

        return sumission

In [5]:
initial_state = [5, 1, 7, 3, 9, 2, 11, 4, 13, 6, 15, 8, None, 10, 14, 12]
#initial_state = [1, 2, 3, 4, 5, 6, None, 8, 9, 10, 7, 11, 13, 14, 15, 12]


game = PuzzleGame(initial_state)

game.matrix = game.generate_matrix(initial_state)
print(game)

if (game.is_solvable()):
   # random.seed(0)
    print("Цель достижима")
    dissimilars = game.count_dissimilar(game.matrix) # кол-во эл-в не на своиъх местах
    place = game.get_block_place(block=None, matrix=game.matrix) #координаты None
    prev_move = None
    level = 0

    ## Initiate game
    while(dissimilars > 0):
        level += 1

        moves = game.check_moves(prev_move) # контроль перемещений
        print("Ход {}: {}".format(level, moves))
        dissimilar = {}
        for move in moves:
            if (game.move_allowed(place, game.MOVE[move])): # если можно переместить
                new = game.new_place(place, game.MOVE[move]) # новая координата нуля
                new_matrix = game.apply_move(place, new) # копируем матрицу

                dissimilars = game.count_dissimilar(new_matrix) 
                dissimilar[move] = dissimilars # кол-во эл-в не на своиъх местах для каждого из перемещений

        print("Варианты передвижений : {}".format(dissimilar))

        dissimilars = min(dissimilar.values())

        dissimilar_moves = list(dissimilar.keys()) # список доступных передвижений
        # random.shuffle(dissimilar_moves)

        for move in dissimilar_moves:
            if dissimilar[move] == dissimilars:
                prev_move = move # куда двигались


        print("Выбраный ход : {}".format(prev_move))

        new_place = game.new_place(place, game.MOVE[prev_move])# новая координата нуля
        game.matrix = game.apply_move(place, new_place) # копируем матрицу
        place = new_place # новые координаты нуля
        print(game)

    print("Общая стоимость пути = {}".format(level - 1))

else:
    print("Цель не достижима")

[5, 1, 7, 3]
[9, 2, 11, 4]
[13, 6, 15, 8]
[None, 10, 14, 12]

Цель достижима
Ход 1: ('UP', 'DOWN', 'LEFT', 'RIGHT')
Варианты передвижений : {'UP': 14, 'RIGHT': 15}
Выбраный ход : UP
[5, 1, 7, 3]
[9, 2, 11, 4]
[None, 6, 15, 8]
[13, 10, 14, 12]

Ход 2: ('UP', 'LEFT', 'RIGHT')
Варианты передвижений : {'UP': 13, 'RIGHT': 14}
Выбраный ход : UP
[5, 1, 7, 3]
[None, 2, 11, 4]
[9, 6, 15, 8]
[13, 10, 14, 12]

Ход 3: ('UP', 'LEFT', 'RIGHT')
Варианты передвижений : {'UP': 12, 'RIGHT': 13}
Выбраный ход : UP
[None, 1, 7, 3]
[5, 2, 11, 4]
[9, 6, 15, 8]
[13, 10, 14, 12]

Ход 4: ('UP', 'LEFT', 'RIGHT')
Варианты передвижений : {'RIGHT': 11}
Выбраный ход : RIGHT
[1, None, 7, 3]
[5, 2, 11, 4]
[9, 6, 15, 8]
[13, 10, 14, 12]

Ход 5: ('UP', 'DOWN', 'RIGHT')
Варианты передвижений : {'DOWN': 10, 'RIGHT': 11}
Выбраный ход : DOWN
[1, 2, 7, 3]
[5, None, 11, 4]
[9, 6, 15, 8]
[13, 10, 14, 12]

Ход 6: ('DOWN', 'LEFT', 'RIGHT')
Варианты передвижений : {'DOWN': 9, 'LEFT': 11, 'RIGHT': 10}
Выбраный ход : DOWN
[1, 2, 7,

# Вывод
Выполнена реализация методов поиска подстроки в строке алоритмами: 1. Кнута-Морриса-Пратта; 2. Упрощенный Бойера-Мура. Оценено время работы каждого алгоритма. Также структурирована работа алгоритма "Пятнашки".